In [291]:
#import dt
from ib_insync import *
import pandas as pd
import asyncio
from IPython.display import display, clear_output
import matplotlib.style
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.6f}'.format
plt.style.use("seaborn-v0_8")
import ib_insync.util as util2
util2.startLoop()
import numpy as np
ib = IB()

In [292]:
connection = ib.connect()

In [293]:
sma_s = 50
sma_l = 200
units=1000

In [294]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)
cfd = CFD('EUR', currency='USD')
ib.qualifyContracts(cfd)
conID=cfd.conId

In [295]:
bars = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='1 D',
    barSizeSetting='1 min',
    whatToShow='MIDPOINT',
    useRTH=True,
    formatDate=2,
    keepUpToDate=True
)
last_bar_date = bars[-1].date

In [296]:
logger = 'something'

def onBarUpdate(bars, hasNewBar):
    global df, last_bar_date
    global logger
    
    logger += 'run'
    
    if bars[-1].date > last_bar_date:
        logger = 'true, btw units={}'.format(units)
        last_bar_date = bars[-1].date
        
        df = pd.DataFrame(bars)[['date', 'open', 'high', 'low', 'close']]
        df.set_index('date', inplace=True)
        
        df['sma_s'] = df.close.rolling(sma_s).mean()
        df['sma_l'] = df.close.rolling(sma_l).mean()
        df.dropna(inplace=True)
        df['position'] = np.where(df['close'] > df['sma_l'], 1, -1)
        
        target = df['position'][-1] * units
        
        logger += 'gonna execute'
        execute_trade(target = target)
        
        clear_output(wait=True)
        display(df)
    else:
        logger +=' false'
        pass
        
def execute_trade(target):
    global current_position, logger
    
    logger += 'im here btw target= {} '.format(target) 
    try: 
        logger+= 'trying to execute'
        current_position = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        logger += 'exception'
        current_position = 0
    
    
    trades = target - current_position
    
    logger += ' | trades = {} | '.format(trades)
    
    if trades > 0:
        logger += 'buy'
        
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    elif trades < 0:
        logger += 'sell'
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        logger += '_nobuynosell_'
        pass
        

In [297]:
bars.updateEvent += onBarUpdate

In [313]:
df = df.drop(columns=['high', 'low', 'open'], axis=1)

In [333]:
df

,open,high,low,close,sma_s,sma_l,position
date,,,,,,,
2024-08-22 00:34:00+00:00,1.115000,1.115100,1.114950,1.115050,1.115320,1.115342,-1
2024-08-22 00:35:00+00:00,1.115050,1.115050,1.115000,1.115000,1.115312,1.115342,-1
2024-08-22 00:36:00+00:00,1.115000,1.115050,1.114950,1.115000,1.115306,1.115341,-1
2024-08-22 00:37:00+00:00,1.115000,1.115100,1.115000,1.115050,1.115300,1.115341,-1
2024-08-22 00:38:00+00:00,1.115050,1.115100,1.115050,1.115050,1.115293,1.115340,-1
...,...,...,...,...,...,...,...
2024-08-22 10:28:00+00:00,1.113750,1.113850,1.113650,1.113700,1.113794,1.114144,-1
2024-08-22 10:29:00+00:00,1.113700,1.113750,1.113650,1.113650,1.113788,1.114136,-1
2024-08-22 10:30:00+00:00,1.113650,1.113850,1.113650,1.113850,1.113786,1.114129,-1


In [316]:
ib.positions()

[Position(account='DUA407556', contract=CFD(conId=143916318, symbol='EUR', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), position=-1000.0, avgCost=1.11165)]

In [387]:
ib.disconnect()

In [360]:
current_position 

1000.0

In [376]:
logger

'true, btw units=1000gonna executeim here btw target= 1000 trying to execute | trades = 0.0 | _nobuynosell_run falserun false'

In [377]:
last_bar_date

datetime.datetime(2024, 8, 22, 11, 3, tzinfo=datetime.timezone.utc)

In [384]:
bars[-1].close

1.114

In [382]:
ib.fills()

[Fill(contract=CFD(conId=143916318, symbol='EUR', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), execution=Execution(execId='000132a9.66c6b817.01.01', time=datetime.datetime(2024, 8, 22, 10, 21, 3, tzinfo=datetime.timezone.utc), acctNumber='DUA407556', exchange='SMART', side='SLD', shares=1000.0, price=1.11365, permId=392629660, clientId=1, orderId=6, liquidation=0, cumQty=1000.0, avgPrice=1.11365, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), commissionReport=CommissionReport(execId='000132a9.66c6b817.01.01', commission=2.0, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 8, 22, 10, 21, 5, 497045, tzinfo=datetime.timezone.utc)),
 Fill(contract=CFD(conId=143916318, symbol='EUR', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), execution=Execution(execId='000132a9.66c6b81c.01.01', time=datetime.datetime(2024, 8, 22, 10, 35, 1, tzinfo=datetime.

In [352]:
util.df([fs.execution for fs in ib.fills()])[['execId', 'time', 'side', 'cumQty', 'avgPrice']].set_index('execId')

,time,side,cumQty,avgPrice
execId,,,,
000132a9.66c6b817.01.01,2024-08-22 10:21:03+00:00,SLD,1000.000000,1.113650
000132a9.66c6b81c.01.01,2024-08-22 10:35:01+00:00,BOT,1000.000000,1.114150
000132a9.66c6b81d.01.01,2024-08-22 10:35:01+00:00,BOT,2000.000000,1.114150


In [381]:
util.df([fs.commissionReport for fs in ib.fills()])[['execId','realizedPNL']].set_index('execId')

,realizedPNL
execId,
000132a9.66c6b817.01.01,0.000000
000132a9.66c6b81c.01.01,-4.500000
000132a9.66c6b81d.01.01,0.000000


In [357]:
#session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize('UTC')

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[['execId', 'time', 'side', 'cumQty', 'avgPrice']].set_index('execId')
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[['execId','realizedPNL']].set_index('execId')
    report = pd.concat([fill_df, profit_df], axis=1).set_index("time")#.loc[session_start:]
    report = report.groupby('time').agg({'side':'first', 'cumQty':'max', 'avgPrice': 'mean', 'realizedPNL':'sum'})
    report['cumPNL'] = report.realizedPNL.cumsum()
    
    clear_output(wait=True)
    try:
        display(df, report)
    except:
        pass

In [385]:
trade_reporting()

,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,
2024-08-22 10:21:03+00:00,SLD,1000.000000,1.113650,0.000000,0.000000
2024-08-22 10:35:01+00:00,BOT,2000.000000,1.114150,-4.500000,-4.500000
2024-08-22 11:08:03+00:00,SLD,2000.000000,1.113950,-2.200000,-6.700000
